| Elemento | Descripción |
|-----------|-------------|
| **Escudos** | <p align="center"><img src="./Imagenes/TECNM.png" width="150"/> <img src="./Imagenes/MSC.png" width="150"/></p> |
| **Maestría** | Maestría en Sistemas Computacionales |
| **Asignatura** | Visión Artificial |
| **Proyecto** | Propiedades y filtros espaciales |
| **Autor** | Alan García Díaz |
| **Fecha de entrega** | 11/10/2025 |

## Introducción

En este experimento se busca **mejorar la calidad de un conjunto de imágenes** mediante diversas técnicas de procesamiento digital.  
Las imágenes seleccionadas son:

- `chest_xray.jpg`  
- `cat.jpg`  
- `forest.jpg`  
- `papyrus.png`

El propósito principal es **evaluar diferentes métodos de ajuste y filtrado** que permitan optimizar la visualización, el contraste y la suavidad de las imágenes.  
Estas técnicas son fundamentales en el procesamiento digital, especialmente en campos como la medicina, visión artificial, fotografía y restauración de imágenes antiguas.

El proceso incluye conversión a escala de grises, análisis estadístico de los valores de intensidad, normalización del contraste y aplicación de filtros espaciales (media, mediana, máximo y mínimo), finalizando con un breve reporte de resultados y conclusiones.


## Enunciado del Problema

A continuación, se presentan los puntos que guían el desarrollo de este trabajo:

### 1. Conversión a Escala de Grises
Mejorar la calidad de las imágenes (después de convertirlas a escala de grises):  
`chest_xray.jpg`, `cat.jpg`, `forest.jpg` y `papyrus.png`.

### 2. Análisis de Intensidades
Determinar el **valor máximo y mínimo de gris** en cada una de las imágenes.

### 3. Normalización del Contraste
Ajustar el **contraste de las imágenes mediante la normalización**, de modo que las intensidades se distribuyan uniformemente entre los valores mínimos y máximos posibles (0–255).

### 4. Filtrado Espacial
Aplicar los siguientes filtros con parámetros libremente definidos:

- **Filtro de la media (promedio):** suaviza el ruido al promediar los valores vecinos.  
- **Filtro de la mediana:** elimina el ruido impulsivo preservando los bordes.

### 5. Función `SubMatriz(img, (x, y), k)`
Implementar una función que devuelva una **submatriz de tamaño k × k píxeles** centrada en las coordenadas `(x, y)` de la imagen.  
La función debe recibir:
- `img`: la imagen original,  
- `(x, y)`: coordenadas del centro,  
- `k`: tamaño impar que define la ventana.

### 6. Filtros Máximo y Mínimo
Usando la función `SubMatriz` del punto anterior, aplicar los **filtros máximo y mínimo** sobre las imágenes.  
Estos filtros permiten resaltar bordes o zonas brillantes/oscuras, respectivamente.

### 7. Reporte Final
Realizar un **reporte con los procedimientos, resultados y conclusiones** de estos experimentos, justificando las decisiones tomadas en cada etapa.

---

## Objetivo General

Mejorar la calidad visual de un conjunto de imágenes aplicando técnicas de procesamiento digital, que incluyan normalización del contraste y distintos filtros espaciales, con el fin de analizar su efecto sobre las características y la interpretación visual de las imágenes.

---

## Objetivos Específicos

1. Convertir las imágenes a escala de grises y calcular sus valores mínimos y máximos de intensidad.  
2. Aplicar una normalización de contraste para mejorar la distribución de niveles de gris.  
3. Implementar y comparar los efectos de los filtros de la media, mediana, máximo y mínimo.  
4. Diseñar una función genérica para obtener submatrices centradas, útil para operaciones locales.  
5. Analizar los resultados obtenidos y formular conclusiones sobre la mejora lograda en las imágenes.


# Mejora de calidad de imágenes (escala de grises)

Este cuaderno realiza los siguientes pasos para cada imagen:
1. Conversión a escala de grises.
2. Cálculo de valores máximo y mínimo de gris.
3. Ajuste de contraste mediante normalización (min-max).
4. Filtrado por media y por mediana (parámetros configurables).
5. Implementación de `SubMatriz(img, (x, y), k)` que extrae una submatriz k×k centrada en (x,y).
6. Aplicación de filtros máximo y mínimo utilizando `SubMatriz` como base.
7. Reporte con procedimientos, resultados y conclusiones.


In [1]:
# IMPORTS Y FUNCIONES AUXILIARES
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from numpy.lib.stride_tricks import sliding_window_view

# Directorio con las imágenes
IMAGE_DIR = "Imagenes"
IMAGES = ["chest_xray.jpg", "cat.jpg", "forest.jpg", "papyrus.png"]

def load_image_gray(path):
    # Carga imagen y la convierte a escala de grises (uint8).
    img = Image.open(path).convert("L")
    arr = np.array(img, dtype=np.uint8)
    return arr

def imshow_ax(ax, img, title=None):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.axis("off")
    if title:
        ax.set_title(title)


In [2]:
# 1) Cálculo de valores máximo y mínimo de gris para cada imagen
minmax_results = {}
for name in IMAGES:
    path = os.path.join(IMAGE_DIR, name)
    if not os.path.exists(path):
        print(f"Archivo no encontrado: {path}  (omite y continúa)")
        continue
    arr = load_image_gray(path)
    mn, mx = int(arr.min()), int(arr.max())
    minmax_results[name] = {"min": mn, "max": mx}
    print(f"{name}: min={mn}, max={mx}")


chest_xray.jpg: min=0, max=255
cat.jpg: min=0, max=255
forest.jpg: min=0, max=251
papyrus.png: min=1, max=255


In [3]:
# 2) Normalización (ajuste de contraste mediante min-max)
def normalize_minmax(img):
    img = img.astype(np.float32)
    mn = img.min()
    mx = img.max()
    if mx == mn:
        return np.clip(img, 0, 255).astype(np.uint8)
    norm = (img - mn) * 255.0 / (mx - mn)
    return np.clip(norm, 0, 255).astype(np.uint8)

# Guardar imágenes normalizadas en carpeta salida
OUT_DIR = "Salida_Procesada"
os.makedirs(OUT_DIR, exist_ok=True)

for name in IMAGES:
    path = os.path.join(IMAGE_DIR, name)
    if not os.path.exists(path):
        continue
    arr = load_image_gray(path)
    norm = normalize_minmax(arr)
    out_path = os.path.join(OUT_DIR, f"norm_{name.replace(' ', '_')}")
    Image.fromarray(norm).save(out_path)
    print(f"Normalizada -> {out_path}  (min={int(norm.min())}, max={int(norm.max())})")


Normalizada -> Salida_Procesada\norm_chest_xray.jpg  (min=0, max=255)
Normalizada -> Salida_Procesada\norm_cat.jpg  (min=0, max=255)
Normalizada -> Salida_Procesada\norm_forest.jpg  (min=0, max=255)
Normalizada -> Salida_Procesada\norm_papyrus.png  (min=0, max=255)


In [4]:
# 3) Filtrado de media y mediana (parámetros libres)
def pad_reflect(img, pad):
    return np.pad(img, pad_width=pad, mode="reflect")

def mean_filter(img, k):
    # aplica filtro de media con ventana k x k, devuelve imagen del mismo tamaño
    if k % 2 == 0 or k < 1:
        raise ValueError("k debe ser impar y >=1")
    pad = k // 2
    padded = pad_reflect(img, pad)
    # sliding window view
    try:
        windows = sliding_window_view(padded, (k, k))
        # windows.shape = (H, W, k, k)
        means = windows.mean(axis=(2,3))
    except Exception:
        # fallback (menos eficiente)
        H, W = img.shape
        means = np.zeros_like(img, dtype=np.float32)
        for i in range(H):
            for j in range(W):
                sub = padded[i:i+k, j:j+k]
                means[i,j] = sub.mean()
    return np.clip(means, 0, 255).astype(np.uint8)

def median_filter(img, k):
    if k % 2 == 0 or k < 1:
        raise ValueError("k debe ser impar y >=1")
    pad = k // 2
    padded = pad_reflect(img, pad)
    try:
        windows = sliding_window_view(padded, (k, k))
        meds = np.median(windows, axis=(2,3))
    except Exception:
        H, W = img.shape
        meds = np.zeros_like(img, dtype=np.float32)
        for i in range(H):
            for j in range(W):
                sub = padded[i:i+k, j:j+k]
                meds[i,j] = np.median(sub)
    return np.clip(meds, 0, 255).astype(np.uint8)

# Ejemplo de uso con k=3 y k=5; guardar resultados
for k in (3,5):
    for name in IMAGES:
        path = os.path.join(IMAGE_DIR, name)
        if not os.path.exists(path):
            continue
        arr = load_image_gray(path)
        m = mean_filter(arr, k)
        med = median_filter(arr, k)
        Image.fromarray(m).save(os.path.join(OUT_DIR, f"mean_k{k}_{name.replace(' ', '_')}"))
        Image.fromarray(med).save(os.path.join(OUT_DIR, f"median_k{k}_{name.replace(' ', '_')}"))
    print(f"Filtros aplicados para k={k} y guardados en {OUT_DIR}")


Filtros aplicados para k=3 y guardados en Salida_Procesada
Filtros aplicados para k=5 y guardados en Salida_Procesada


In [5]:
# 4) Función SubMatriz(img, (x,y), k)
# Esta implementación usa padding reflectante para poder obtener una submatriz kxk
# centrada en cualquier coordenada (x,y) de la imagen.
def SubMatriz(img, center, k):
    if k % 2 == 0 or k < 1:
        raise ValueError("k debe ser impar y >=1")
    cx, cy = center
    H, W = img.shape
    pad = k // 2
    # pad image by reflect so any center is valid
    padded = np.pad(img, pad_width=pad, mode="reflect")
    # shift center to padded coordinates
    px = cx + pad
    py = cy + pad
    sub = padded[py-pad:py+pad+1, px-pad:px+pad+1]
    return sub.copy()


In [6]:
# 5) Filtros máximo y mínimo usando SubMatriz
def max_filter_using_submatrix(img, k):
    H, W = img.shape
    out = np.zeros_like(img)
    for y in range(H):
        for x in range(W):
            sub = SubMatriz(img, (x,y), k)
            out[y,x] = sub.max()
    return out

def min_filter_using_submatrix(img, k):
    H, W = img.shape
    out = np.zeros_like(img)
    for y in range(H):
        for x in range(W):
            sub = SubMatriz(img, (x,y), k)
            out[y,x] = sub.min()
    return out

# Ejecutar para k=3 y guardar
for k in (3,5):
    for name in IMAGES:
        path = os.path.join(IMAGE_DIR, name)
        if not os.path.exists(path):
            continue
        arr = load_image_gray(path)
        mx = max_filter_using_submatrix(arr, k)
        mn = min_filter_using_submatrix(arr, k)
        Image.fromarray(mx).save(os.path.join(OUT_DIR, f"max_k{k}_{name.replace(' ', '_')}"))
        Image.fromarray(mn).save(os.path.join(OUT_DIR, f"min_k{k}_{name.replace(' ', '_')}"))
    print(f"Filtros máximo/mínimo con k={k} guardados en {OUT_DIR}")


Filtros máximo/mínimo con k=3 guardados en Salida_Procesada
Filtros máximo/mínimo con k=5 guardados en Salida_Procesada


## 6) Reporte — Procedimientos, resultados y conclusiones

### Procedimientos
- Se convirtieron las imágenes a escala de grises usando `Pillow` para asegurar una base uniforme.
- Se calcularon los valores mínimo y máximo de gris en cada imagen para identificar el rango dinámico.
- Se normalizó cada imagen con la técnica min-max: `I' = (I - min) * 255 / (max - min)` para aprovechar todo el rango 0–255.
- Se aplicaron filtros de media y mediana con ventanas configurables (ej. k=3,5). La media suaviza el ruido gaussiano, la mediana preserva bordes y elimina mejor el ruido impulsivo (salt-and-pepper).
- Se implementó `SubMatriz(img,(x,y),k)` con padding reflectante para extraer ventanas centradas en cualquier píxel.
- Usando `SubMatriz` se implementaron los filtros máximo y mínimo (útiles para operaciones morfológicas básicas: dilatación y erosión en imágenes en escala de grises).

### Resultados esperados y guardados
- Las imágenes normalizadas (`norm_*`) muestran mayor rango de contraste cuando la imagen original tenía rango comprimido.
- `mean_k*` contienen versiones suavizadas; cuanto mayor `k`, más suavizado (aunque pérdida de detalle).
- `median_k*` preservan bordes y eliminan puntos aislados.
- `max_k*` tienden a resaltar brillancias locales (similar a dilatación).
- `min_k*` atenúan regiones brillantes y realzan sombras locales (similar a erosión).

### Conclusiones y recomendaciones
- **Normalización (min-max)** es un ajuste simple y efectivo cuando la imagen no está correctamente aprovechando el rango dinámico. No obstante, en casos con ruido intenso o con contrastes locales heterogéneos, es preferible usar contrast limited adaptive histogram equalization (CLAHE) — técnica avanzada que debe evaluarse si necesita mejorar contraste local sin amplificar ruido.
- **Filtro de media** es útil para reducir ruido suave pero degradará bordes. Úselo cuando la prioridad sea reducción de ruido global y no la preservación de detalles finos.
- **Filtro de mediana** es recomendado si existe ruido impulsivo (pixeles salt-and-pepper) ya que preserva bordes mejor.
- **Filtros máximo y mínimo** (operaciones morfológicas por intensidad) son útiles para eliminar pequeñas manchas oscuras o pequeñas partículas brillantes, y para dar efecto de realce o erosión según sea necesario.
- Para producción o procesamiento en lote de muchas imágenes grandes, es preferible usar implementaciones optimizadas (OpenCV, scipy.ndimage) por eficiencia.

---
> **Nota:** El cuaderno guarda todas las imágenes procesadas en la carpeta `Salida_Procesada`.